In [2]:
import pandas as pd
import string
import random
import numpy as np

In [ ]:
food_df = pd.read_csv('us_food_data_filtered.csv')

In [ ]:
import re

def extract_numeric_serving_size(serving_size):
    
    """
    Extracts numeric serving sizes from a string and converts them to grams.

    Args:
        serving_size (string): The serving size string to be processed.

    Returns:
        float: The serving size in grams. Defaults to 1 gram if no valid serving size is found.
    """

    serving_size_str = str(serving_size).lower()

    # First, look for values specified in grams (g)
    gram_match = re.search(r'(\d+\.?\d*)\s*g', serving_size_str)
    if gram_match:
        return float(gram_match.group(1))

    # If no gram value is found, then look for other units and convert them to grams
    conversion_factors = {
        'ml': 1,  # Assuming 1 ml of water is approximately 1 gram
        'oz': 28.3495,  # 1 ounce is approximately 28.3495 grams
        'cup': 240  # 1 cup is approximately 240 grams (for water-based items)
    }

    # Extract the numeric value and unit
    match = re.search(r'(\d+\.?\d*)\s*(ml|oz|cup)?', serving_size_str)
    if match:
        value = float(match.group(1))
        unit = match.group(2)

        # Convert to grams if necessary
        if unit in conversion_factors:
            return value * conversion_factors[unit]
        elif unit is None:
            return value

    return 1.0  # Default to 1 gram if no match found or unable to convert


food_df['num_serving_size'] = food_df['serving_size'].apply(lambda x: extract_numeric_serving_size(x))


In [807]:
import ipywidgets as widgets
from IPython.display import display
from macro_calculator import basic_metabolic_rate, total_daily_energy_expenditure, diet_goals, macro_converter 

# Widgets definition
age_widget = widgets.IntText(min=18, max=100, value=25, step=1, description='Age:')
metric_system = widgets.RadioButtons(
    options=['Imperial', 'International System'],
    value='Imperial',
    description='<div style="text-align:left;">Metric System:</div>',  # Use HTML to format the label
    layout={'width': 'auto'}  # Adjust the width as needed
)
weight_widget = widgets.FloatText(min=80, max=300, value=150, step=0.5, description='Weight (lb):')
height_widget = widgets.FloatText(min=54, max=84, value=66, step=0.5, description='Height (in):')

# Function to update weight and height widgets based on metric system selection
def update_widgets(change):
    weight_widget.min, weight_widget.max = 0, 1000
    height_widget.min, height_widget.max = 0, 1000
    if change['new'] == 'Imperial':
        weight_widget.value, height_widget.value = 150, 66
        weight_widget.min, weight_widget.max = 80, 300
        weight_widget.description = 'Weight (lb):'
        height_widget.min, height_widget.max = 54, 84
        height_widget.description = 'Height (in):'
    else:
        weight_widget.value, height_widget.value = 68, 167
        weight_widget.min, weight_widget.max = 36, 136
        weight_widget.description = 'Weight (kg):'
        height_widget.min, height_widget.max = 137, 213
        height_widget.description = 'Height (cm):'

metric_system.observe(update_widgets, names='value')

diet_widget = widgets.Dropdown(
    options=['Balanced', 'Keto', 'High Protein', 'Low Fat'],
    value='Balanced',
    description='Diet:'
)
diet_goal_widget = widgets.Dropdown( 
    options=['Lose', 'Maintain', 'Gain'],
    value='Maintain',
    description='Weight Goals:'
)
gender = widgets.RadioButtons(
    options=['Male', 'Female'],
    value='Male',
    description='Gender'
)
activity_level = widgets.Dropdown(
    options=['Sedentary', 'Lightly Active', 'Moderately Active', 'Very Active', 'Super Active'],
    value='Moderately Active',
    description='Activity Level: '
)
submit_button = widgets.Button(description="Submit")

# Global variable to store user input
user_input = None

# Function to handle button clicks
def on_submit_button_clicked(b):
    global user_input  # Declare global variable to update it
    metric_system_value = metric_system.value 
    age_value = int(age_widget.value)
    height_value = float(height_widget.value)
    weight_value = float(weight_widget.value)
    diet_type_value = diet_widget.value 
    diet_goal_value = diet_goal_widget.value
    activity_level_value = activity_level.value 
    gender_value = gender.value.lower()
    user_input = diet_type_value  # Update the global user_input variable
    process_user_input(metric_system_value, age_value, height_value, weight_value, diet_type_value, diet_goal_value, activity_level_value, gender_value)

submit_button.on_click(on_submit_button_clicked)

# Function to process the input values
def process_user_input(metric_system, age, height, weight, diet_type, diet_goal, activity_level, gender):
    global global_macro_convert
    print(f"Metric System: {metric_system}, Age: {age}, Height: {height}, Weight: {weight}, Diet Type: {diet_type}, Diet Goal: {diet_goal}, Activity Level: {activity_level}, Gender: {gender}")
    bmr = basic_metabolic_rate(age=age, weight=weight, height=height, gender=gender, metric_type=metric_system)
    tdee = total_daily_energy_expenditure(activity_level=activity_level, BMR=bmr)
    macro_breakdown = diet_goals(goal=diet_goal, diet_type=diet_type)
    macro_convert = macro_converter(tdee, macro_breakdown=macro_breakdown)
    global_macro_convert = macro_convert

# Display all widgets including the button
display(metric_system, age_widget, height_widget, weight_widget, diet_widget, diet_goal_widget, activity_level, gender, submit_button)


RadioButtons(description='<div style="text-align:left;">Metric System:</div>', layout=Layout(width='auto'), op…

IntText(value=25, description='Age:')

FloatText(value=66.0, description='Height (in):', step=0.5)

FloatText(value=150.0, description='Weight (lb):', step=0.5)

Dropdown(description='Diet:', options=('Balanced', 'Keto', 'High Protein', 'Low Fat'), value='Balanced')

Dropdown(description='Weight Goals:', index=1, options=('Lose', 'Maintain', 'Gain'), value='Maintain')

Dropdown(description='Activity Level: ', index=2, options=('Sedentary', 'Lightly Active', 'Moderately Active',…

RadioButtons(description='Gender', options=('Male', 'Female'), value='Male')

Button(description='Submit', style=ButtonStyle())

Metric System: Imperial, Age: 25, Height: 66.0, Weight: 150.0, Diet Type: High Protein, Diet Goal: Maintain, Activity Level: Moderately Active, Gender: male
Metric System: Imperial, Age: 25, Height: 66.0, Weight: 150.0, Diet Type: Balanced, Diet Goal: Maintain, Activity Level: Moderately Active, Gender: male


Clean data based off of user input

In [822]:
def filter_diet_types(diet_input, df):
    #get diet types that match what user inputted above
    temp_vals = df['final_diet_types'].apply(lambda x: diet_input in x)
    final_df = df[temp_vals]
    return final_df 

filtered_food_df = filter_diet_types(user_input, food_df)

In [823]:
macro_allocation = {
    'Balanced': {
        'breakfast': {'Carbohydrates': 0.2, 'Protein': 0.2, 'Fats': 0.2},
        'lunch': {'Carbohydrates': 0.3, 'Protein': 0.3, 'Fats': 0.3},
        'dinner': {'Carbohydrates': 0.4, 'Protein': 0.4, 'Fats': 0.4},
        'snack': {'Carbohydrates': 0.1, 'Protein': 0.1, 'Fats': 0.1}
    },
    'Keto': {
        'breakfast': {'Carbohydrates': 0.025, 'Protein': 0.25, 'Fats': 0.25},
        'lunch': {'Carbohydrates': 0.025, 'Protein': 0.25, 'Fats': 0.25},
        'dinner': {'Carbohydrates': 0.04, 'Protein': 0.4, 'Fats': 0.4},
        'snack': {'Carbohydrates': 0.01, 'Protein': 0.1, 'Fats': 0.1}
    },
    'High Protein': {
        'breakfast': {'Carbohydrates': 0.2, 'Protein': 0.2, 'Fats': 0.2},
        'lunch': {'Carbohydrates': 0.35, 'Protein': 0.35, 'Fats': 0.35},
        'dinner': {'Carbohydrates': 0.35, 'Protein': 0.35, 'Fats': 0.35},
        'snack': {'Carbohydrates': 0.1, 'Protein': 0.1, 'Fats': 0.1}
    },
    'Low Fat': {
        'breakfast': {'Carbohydrates': 0.2, 'Protein': 0.2, 'Fats': 0.1},
        'lunch': {'Carbohydrates': 0.3, 'Protein': 0.3, 'Fats': 0.2},
        'dinner': {'Carbohydrates': 0.4, 'Protein': 0.4, 'Fats': 0.3},
        'snack': {'Carbohydrates': 0.1, 'Protein': 0.1, 'Fats': 0.1}
    },
    'Vegetarian': {
        'breakfast': {'Carbohydrates': 0.2, 'Protein': 0.2, 'Fats': 0.2},
        'lunch': {'Carbohydrates': 0.3, 'Protein': 0.3, 'Fats': 0.3},
        'dinner': {'Carbohydrates': 0.4, 'Protein': 0.4, 'Fats': 0.4},
        'snack': {'Carbohydrates': 0.1, 'Protein': 0.1, 'Fats': 0.1}
    },
    'Vegan': {
        'breakfast': {'Carbohydrates': 0.2, 'Protein': 0.2, 'Fats': 0.2},
        'lunch': {'Carbohydrates': 0.3, 'Protein': 0.3, 'Fats': 0.3},
        'dinner': {'Carbohydrates': 0.4, 'Protein': 0.4, 'Fats': 0.4},
        'snack': {'Carbohydrates': 0.1, 'Protein': 0.1, 'Fats': 0.1}
    },
    'Paleo': {
        'breakfast': {'Carbohydrates': 0.25, 'Protein': 0.25, 'Fats': 0.25},
        'lunch': {'Carbohydrates': 0.3, 'Protein': 0.3, 'Fats': 0.3},
        'dinner': {'Carbohydrates': 0.35, 'Protein': 0.35, 'Fats': 0.35},
        'snack': {'Carbohydrates': 0.1, 'Protein': 0.1, 'Fats': 0.1}
    }
}

def allocate_macros(user_macros, diet_type, macro_allocation):
    #create dictionary on macro percentages by meal 
    allocated_macros = {}
    for meal, percentages in macro_allocation[diet_type].items():
        allocated_macros[meal] = {macro: round(user_macros[macro] * percentages[macro],0) for macro in user_macros}
    return allocated_macros


allocated_macros = allocate_macros(global_macro_convert, user_input, macro_allocation)


In [824]:
def is_high_carb(food):
    #create a dataframe to get foods that are high in carbs 
    meal_keywords = ['nut', 'yogurt', 'rice', 'pasta', 'noodle', 'spinach', 'broccoli', 'pea', 'lentil']
    fats = food['fat_100g']
    carbohydrates = food['carbohydrates_100g']
    proteins = food['proteins_100g']
    total_macros = fats + carbohydrates + proteins

    if total_macros == 0:
        return False

    carbs_ratio = carbohydrates / total_macros
    
    # Check if the food's name contains any of the keywords
    food_name = food['product_name'].lower()
    if any(keyword in food_name for keyword in meal_keywords):
        return carbs_ratio > 0.5
    
    return False

# Filter high-carb foods
high_carb_foods = food_df[food_df.apply(is_high_carb, axis=1)]


In [861]:
def filter_foods_for_diet(food_df, diet_type, tolerance):
    #function that filters foods out based off of their macros and if it fits the user inputs or not 
    macro_ratios = macro_allocation[diet_type]

    def meets_ratio_criteria(row, ratio):
        fats = row['fat_100g']
        carbohydrates = row['carbohydrates_100g']
        proteins = row['proteins_100g']
        total_macros = fats + carbohydrates + proteins

        if total_macros == 0:
            return False

        fats_ratio = fats / total_macros
        carbs_ratio = carbohydrates / total_macros
        protein_ratio = proteins / total_macros

        return ((ratio['Carbohydrates'] - tolerance <= carbs_ratio <= ratio['Carbohydrates'] + tolerance or carbs_ratio == 0) and
                ratio['Protein'] - tolerance - 1 <= protein_ratio <= ratio['Protein'] + tolerance + 1 and
                (ratio['Fats'] - tolerance <= fats_ratio <= ratio['Fats'] + tolerance or fats_ratio == 0))

    def meets_high_protein_criteria(row):
        fats = row['fat_100g']
        carbohydrates = row['carbohydrates_100g']
        proteins = row['proteins_100g']
        total_macros = fats + carbohydrates + proteins

        if total_macros == 0:
            return False

        protein_ratio = proteins / total_macros

        return protein_ratio >= 0.5

    def meets_balanced_criteria(row):
        fats = row['fat_100g']
        carbohydrates = row['carbohydrates_100g']
        proteins = row['proteins_100g']
        total_macros = fats + carbohydrates + proteins

        if total_macros == 0:
            return False

        fats_ratio = fats / total_macros
        carbs_ratio = carbohydrates / total_macros
        protein_ratio = proteins / total_macros

        return (0 <= carbs_ratio <= 0.45 and
                0.1 <= protein_ratio <= 0.6 and
                0.1 <= fats_ratio <= 0.4)

    def meets_keto_criteria(row):
        fats = row['fat_100g']
        carbohydrates = row['carbohydrates_100g']
        proteins = row['proteins_100g']
        total_macros = fats + carbohydrates + proteins

        if total_macros == 0:
            return False

        fats_ratio = fats / total_macros
        carbs_ratio = carbohydrates / total_macros
        protein_ratio = proteins / total_macros

        return (carbs_ratio <= 0.2 and
                protein_ratio >= 0.25 and
                fats_ratio >= 0.25)

    def meets_low_fat_criteria(row):
        fats = row['fat_100g']
        carbohydrates = row['carbohydrates_100g']
        proteins = row['proteins_100g']
        total_macros = fats + carbohydrates + proteins

        if total_macros == 0:
            return False

        fats_ratio = fats / total_macros
        carbs_ratio = carbohydrates / total_macros
        protein_ratio = proteins / total_macros

        return fats_ratio <= 0.2

    def food_meets_any_meal_criteria(row, ratios):
        return any(meets_ratio_criteria(row, ratio) for ratio in ratios.values())

    if diet_type == 'High Protein':
        filtered_df = food_df[food_df.apply(meets_high_protein_criteria, axis=1)]
    elif diet_type == 'Balanced':
        filtered_df = food_df[food_df.apply(meets_balanced_criteria, axis=1)]
    elif diet_type == 'Keto':
        filtered_df = food_df[food_df.apply(meets_keto_criteria, axis=1)]
    elif diet_type == 'Low Fat':
        filtered_df = food_df[food_df.apply(meets_low_fat_criteria, axis=1)]
    else:
        filtered_df = food_df[food_df.apply(lambda row: food_meets_any_meal_criteria(row, macro_ratios), axis=1)]

    return filtered_df
# Filter foods for the 'Balanced' diet type
filtered_food_df = filter_foods_for_diet(food_df, user_input, tolerance=.1)


In [862]:
def filter_foods(food_type, df):
    #filter foods based off of meal category 
    temp_vals = df['meal_category'].apply(lambda x: food_type in x)
    final_df = df[temp_vals]
    return final_df 

filtered_food_breakfast_df = filter_foods('breakfast', filtered_food_df)
filtered_food_breakfast_df.reset_index(drop=True, inplace=True)

filtered_food_lunch_df = filter_foods('lunch', filtered_food_df)
filtered_food_lunch_df.reset_index(drop=True, inplace=True)

filtered_food_dinner_df = filter_foods('dinner', filtered_food_df)
filtered_food_dinner_df.reset_index(drop=True, inplace=True)

filtered_food_snacks_df = filter_foods('snack', filtered_food_df)
filtered_food_snacks_df.reset_index(drop=True, inplace=True)

columns=['code', 'product_name', 'categories_en', 'serving_size', 'fat_100g', 'carbohydrates_100g', 'proteins_100g', 'meal_category', 'final_diet_types', 'cluster_labels', 'num_serving_size']

calculate_macro_df = filtered_food_df.copy()

calculate_macro_df = calculate_macro_df[columns]

breakfast_df = filtered_food_breakfast_df.copy()
breakfast_df = breakfast_df[columns]

lunch_df = filtered_food_lunch_df.copy()
lunch_df = lunch_df[columns]

dinner_df = filtered_food_dinner_df.copy()
dinner_df = dinner_df[columns]

snacks_df = filtered_food_snacks_df.copy()
snacks_df = snacks_df[columns]

meal_plan_list = {
    'breakfast': breakfast_df,
    'lunch': lunch_df,
    'dinner': dinner_df,
    'snack': snacks_df  
}

Function to decide what meal to choose for user

In [863]:
def calculate_score(food, target_ratios, target_macros):
    """
    Calculates a score for a food item based on its macronutrient composition compared to target ratios and target macros.

    Args:
        food (dict): A dictionary containing the macronutrient values for a food item.
            Expected keys are 'carbohydrates_100g', 'proteins_100g', and 'fat_100g'.
        target_ratios (dict): A dictionary containing the target macronutrient ratios.
            Expected keys are 'Carbohydrates', 'Protein', and 'Fats'.
        target_macros (dict): A dictionary containing the target macronutrient values.
            Expected keys are 'Carbohydrates', 'Protein', and 'Fats'.

    Returns:
        float: The calculated score for the food item.
    """

    weights = {'Carbohydrates': 1.0, 'Protein': 0.5, 'Fats': 1.0}  # Adjusted protein weight
    
    # Calculate total macros
    total_macros = food['carbohydrates_100g'] + food['proteins_100g'] + food['fat_100g']
    if total_macros == 0:
        return float('inf')
    
    # Calculate food ratio
    food_ratio = {
        'Carbohydrates': food['carbohydrates_100g'] / total_macros,
        'Protein': food['proteins_100g'] / total_macros,
        'Fats': food['fat_100g'] / total_macros,
    }
    
    # Calculate ratio score with weights
    ratio_score = (
        weights['Carbohydrates'] * abs(target_ratios['Carbohydrates'] - food_ratio['Carbohydrates']) +
        weights['Protein'] * abs(target_ratios['Protein'] - food_ratio['Protein']) +
        weights['Fats'] * abs(target_ratios['Fats'] - food_ratio['Fats'])
    )
    
    # Check for zero values in target macros to avoid division by zero
    carbohydrate_score = weights['Carbohydrates'] * abs(target_macros['Carbohydrates'] - food['carbohydrates_100g']) / target_macros['Carbohydrates'] if target_macros['Carbohydrates'] != 0 else 0
    protein_score = weights['Protein'] * abs(target_macros['Protein'] - food['proteins_100g']) / target_macros['Protein'] if target_macros['Protein'] != 0 else 0
    fat_score = weights['Fats'] * abs(target_macros['Fats'] - food['fat_100g']) / target_macros['Fats'] if target_macros['Fats'] != 0 else 0
    
    # Calculate absolute deviation score with weights
    absolute_deviation_score = carbohydrate_score + protein_score + fat_score
    
    # Combine both scores
    total_score = ratio_score + absolute_deviation_score
    
    return total_score


In [871]:
def create_weighted_meal_plan(meal_plan_list, meal_macros, diet_type):
    """
    Creates a weighted meal plan based on the provided macros and diet type.

    Args:
        meal_plan_list (dict): A dictionary where keys are meal types and values are DataFrames containing food items.
        meal_macros (dict): A dictionary where keys are meal types and values are dictionaries of target macronutrients for each meal.
        diet_type (string): The diet type to filter food items by (e.g., 'vegan', 'keto').

    Returns:
        dict: A dictionary representing the meal plan, where keys are meal types and values are lists of selected food items.
    """

    meal_plan = {}
    selected_foods = set()  # To track already selected foods

    for meal, macros in meal_macros.items():
        meal_plan[meal] = []
    
        df = meal_plan_list[meal].copy()

        df = df[df['final_diet_types'].apply(lambda x: diet_type in x) & ((df['carbohydrates_100g'] > 0) | (df['proteins_100g'] > 0) | (df['fat_100g'] > 0))]
        
        if df.empty:
            continue

        df['score'] = df.apply(lambda x: calculate_score(x, macros, meal_macros[meal]), axis=1)

        top_10_percentile_threshold = df['score'].quantile(1)
        top_10_percentile_foods = df[df['score'] <= top_10_percentile_threshold]

        if top_10_percentile_foods.empty:
            continue

        selected_food = top_10_percentile_foods.sample(n=1).iloc[0]

        serving_size_numeric = selected_food['num_serving_size']
        adjusted_carb = selected_food['carbohydrates_100g']
        adjusted_protein = selected_food['proteins_100g']
        adjusted_fat = selected_food['fat_100g']

        food_item = {
            'product_name': selected_food['product_name'],
            'carbohydrates': adjusted_carb,
            'proteins': adjusted_protein,
            'fats': adjusted_fat,
            'serving_size': serving_size_numeric
        }
        
        meal_plan[meal].append(food_item)
        
        selected_foods.add(selected_food['product_name'])

    return meal_plan

In [872]:
def adjust_serving_size(food, max_macros):
    """
    Adjusts the serving size of a food item to ensure it meets the maximum allowed macronutrient values.

    Args:
        food (dict): A dictionary containing the macronutrient values and serving size for a food item.
            Expected keys are 'product_name', 'carbohydrates', 'proteins', 'fats', and 'serving_size'.
        max_macros (dict): A dictionary containing the maximum allowed macronutrient values.
            Expected keys are 'Carbohydrates', 'Protein', and 'Fats'.

    Returns:
        dict: A dictionary representing the adjusted food item with scaled macronutrient values and serving size.
            If the food item does not contain any macronutrients, it returns the original food item.
    """

    limiting_factors = []
    if food['carbohydrates'] > 0:
        limiting_factors.append(max_macros['Carbohydrates'] / food['carbohydrates'])
    if food['proteins'] > 0:
        limiting_factors.append(max_macros['Protein'] / food['proteins'])
    if food['fats'] > 0:
        limiting_factors.append(max_macros['Fats'] / food['fats'])
    
    # Ensure that limiting_factors is not empty
    if not limiting_factors:
        return food
    
    limiting_factor = min(limiting_factors)
    
    # Scale the food's macronutrients and serving size
    adjusted_food = {
        'product_name': food['product_name'],
        'carbohydrates': food['carbohydrates'] * limiting_factor,
        'proteins': food['proteins'] * limiting_factor,
        'fats': food['fats'] * limiting_factor,
        'serving_size': food['serving_size'] * limiting_factor
    }

    return adjusted_food

def adjust_meal_plan_servings(meal_plan, allocated_macros):
    """
    Adjusts the serving sizes of all food items in a meal plan to ensure they meet the allocated macronutrient values for each meal.

    Args:
        meal_plan (dict): A dictionary representing the meal plan, where keys are meal types (e.g., 'breakfast', 'lunch', 'dinner')
                        and values are lists of food items. Each food item is a dictionary containing the macronutrient values 
                        and serving size.
        allocated_macros (dict): A dictionary where keys are meal types and values are dictionaries of the allocated macronutrient 
                                values for each meal. Each macronutrient dictionary should have the keys 'Carbohydrates', 'Protein', 
                                and 'Fats'.

    Returns:
        dict: A dictionary representing the adjusted meal plan, where keys are meal types and values are lists of adjusted food 
            items with scaled macronutrient values and serving sizes.
    """

    adjusted_meal_plan = {}
    for meal, foods in meal_plan.items():
        adjusted_meal_plan[meal] = []
        for food in foods:
            adjusted_food = adjust_serving_size(food, allocated_macros[meal])
            adjusted_meal_plan[meal].append(adjusted_food)
    return adjusted_meal_plan

In [873]:
def calculate_final_macros(meal_plan):
    """
    Calculates the total macronutrients for the entire meal plan and updates each food item with final rounded macronutrient values.

    Args:
        meal_plan (dict): A dictionary representing the meal plan, where keys are meal types (e.g., 'breakfast', 'lunch', 'dinner')
                        and values are lists of food items. Each food item is a dictionary containing the macronutrient values 
                        and serving size.

    Returns:
        tuple: A tuple containing the total carbohydrates, total protein, and total fats for the entire meal plan.
    """

    total_carbs = 0
    total_protein = 0
    total_fats = 0

    for meal, foods in meal_plan.items():
        for food in foods:
            food['final_carbohydrates'] = np.round(food['carbohydrates'], 2)
            food['final_proteins'] = np.round(food['proteins'], 2)
            food['final_fats'] = np.round(food['fats'], 2)
            food['adjusted_serving_size'] = np.round(food['serving_size'])
            total_carbs += food['final_carbohydrates']
            total_protein += food['final_proteins']
            total_fats += food['final_fats']

    return total_carbs, total_protein, total_fats


In [874]:
def iterative_adjustment(meal_plan_list, allocated_macros, diet_type, tolerance=0.2, max_iterations=15):
    """
    Iteratively adjusts a meal plan to meet allocated macronutrient goals within a specified tolerance.

    Args:
        meal_plan_list (dict): A dictionary where keys are meal types (e.g., 'breakfast', 'lunch', 'dinner')
                            and values are DataFrames containing food items.
        allocated_macros (dict): A dictionary where keys are meal types and values are dictionaries of target
                                macronutrient values for each meal. Each macronutrient dictionary should have the keys
                                'Carbohydrates', 'Protein', and 'Fats'.
        diet_type (str): The diet type to filter food items by (e.g., 'vegan', 'keto').
        tolerance (float): The tolerance within which the total macronutrient values should fall relative to the target
                        values. Default is 0.2 (20%).
        max_iterations (int): The maximum number of iterations to perform. Default is 15.

    Returns:
        dict: The best meal plan that meets the allocated macronutrient goals within the specified tolerance.
            If no meal plan meets the goals within the maximum number of iterations, returns the closest meal plan.
    """

    iteration = 0
    within_tolerance = False
    best_plan = None
    best_diff = float('inf')

    while not within_tolerance and iteration < max_iterations:
        iteration += 1
        # Create initial meal plan
        meal_plan = create_weighted_meal_plan(meal_plan_list, allocated_macros, diet_type)

        # Adjust the meal plan servings
        adjusted_meal_plan = adjust_meal_plan_servings(meal_plan, allocated_macros)

        # Calculate final macros
        total_carbs, total_protein, total_fats = calculate_final_macros(adjusted_meal_plan)

        # Calculate the differences from the target macros
        carbs_diff = abs(total_carbs - sum(allocated_macros[meal]['Carbohydrates'] for meal in allocated_macros))
        protein_diff = abs(total_protein - sum(allocated_macros[meal]['Protein'] for meal in allocated_macros))
        fats_diff = abs(total_fats - sum(allocated_macros[meal]['Fats'] for meal in allocated_macros))

        # Calculate the total difference
        total_diff = carbs_diff + protein_diff + fats_diff

        # Check if the results are within tolerance
        carbs_within_tolerance = carbs_diff / sum(allocated_macros[meal]['Carbohydrates'] for meal in allocated_macros) <= tolerance
        protein_within_tolerance = protein_diff / sum(allocated_macros[meal]['Protein'] for meal in allocated_macros) <= tolerance
        fats_within_tolerance = fats_diff / sum(allocated_macros[meal]['Fats'] for meal in allocated_macros) <= tolerance

        within_tolerance = carbs_within_tolerance and protein_within_tolerance and fats_within_tolerance

        if within_tolerance:
            best_plan = adjusted_meal_plan
            break
        else:
            if total_diff < best_diff:
                best_diff = total_diff
                best_plan = adjusted_meal_plan

    return best_plan



In [875]:
def calculate_macros_per_meal(meal_plan):
    """
    Calculates the total macronutrients for each meal in a meal plan.

    Args:
        meal_plan (dict): A dictionary representing the meal plan, where keys are meal types (e.g., 'breakfast', 'lunch', 'dinner')
                        and values are lists of food items. Each food item is a dictionary containing macronutrient values
                        with keys 'carbohydrates', 'proteins', and 'fats'.

    Returns:
        dict: A dictionary where keys are meal types and values are dictionaries of the total macronutrients for each meal.
            Each macronutrient dictionary contains keys 'Carbohydrates', 'Protein', and 'Fats'.
    """

    meal_macros = {}
    for meal, foods in meal_plan.items():
        total_carbs = sum(food['carbohydrates'] for food in foods)
        total_protein = sum(food['proteins'] for food in foods)
        total_fats = sum(food['fats'] for food in foods)
        meal_macros[meal] = {'Carbohydrates': total_carbs, 'Protein': total_protein, 'Fats': total_fats}
    return meal_macros

def add_high_carb_food_to_meal(meal_plan, high_carb_df, meal_deltas):
    """
    Adds high-carbohydrate food items to the specified meals in a meal plan based on the provided carbohydrate deficits.

    Args:
        meal_plan (dict): A dictionary representing the meal plan, where keys are meal types (e.g., 'breakfast', 'lunch', 'dinner')
                        and values are lists of food items. Each food item is a dictionary containing macronutrient values
                        with keys 'carbohydrates', 'proteins', 'fats', and 'serving_size'.
        high_carb_df (pandas.DataFrame): A DataFrame containing high-carbohydrate food items. Expected columns include
                                        'product_name', 'carbohydrates_100g', 'proteins_100g', and 'fat_100g'.
        meal_deltas (dict): A dictionary where keys are meal types and values are dictionaries of macronutrient deficits for
                            each meal. Each macronutrient dictionary contains keys 'Carbohydrates', 'Protein', and 'Fats'.

    Returns:
        dict: The updated meal plan with high-carbohydrate food items added to the specified meals to meet the carbohydrate deficits.
    """

    for meal, delta in meal_deltas.items():
        if meal != 'snack' and delta['Carbohydrates'] > 0 and delta['Protein'] <= 0 and delta['Fats'] > 0:
            # Sample a high-carb food from the dataframe
            high_carb_food = high_carb_df.sample(n=1).iloc[0]
            # Calculate the serving size needed to meet the carb delta
            serving_size = (delta['Carbohydrates'] / high_carb_food['carbohydrates_100g']) * 100
            # Calculate the actual carbs, proteins, and fats for the adjusted serving size
            actual_carbs = high_carb_food['carbohydrates_100g'] * (serving_size / 100)
            actual_proteins = high_carb_food['proteins_100g'] * (serving_size / 100)
            actual_fats = high_carb_food['fat_100g'] * (serving_size / 100)
            
            # Add the high-carb food to the meal
            food_item = {
                'product_name': high_carb_food['product_name'],
                'carbohydrates': actual_carbs,
                'proteins': actual_proteins,
                'fats': actual_fats,
                'serving_size': serving_size
            }
            # Append the high-carb food item to the meal plan
            meal_plan[meal].append(food_item)
    return meal_plan

def adjust_serving_sizes_protein(meal_plan, excess_protein):
    """
    Adjusts the serving sizes of food items in a meal plan to reduce excess protein evenly between specified meals.

    Args:
        meal_plan (dict): A dictionary representing the meal plan, where keys are meal types (e.g., 'breakfast', 'lunch', 'dinner')
                        and values are lists of food items. Each food item is a dictionary containing macronutrient values
                        with keys 'carbohydrates', 'proteins', 'fats', and 'serving_size'.
        excess_protein (float): The total amount of excess protein that needs to be reduced from the meal plan.

    Returns:
        dict: The adjusted meal plan with serving sizes reduced to decrease the excess protein.
    """

    adjusted_meal_plan = meal_plan.copy()
    meals_to_adjust = ['lunch', 'dinner']
    protein_to_reduce = excess_protein / 2

    for meal in meals_to_adjust:
        total_meal_protein = sum(food['proteins'] for food in adjusted_meal_plan[meal])
        if total_meal_protein > 0:
            for food in adjusted_meal_plan[meal]:
                protein_contribution = food['proteins'] / total_meal_protein
                protein_reduction = protein_contribution * protein_to_reduce
                reduction_ratio = (food['proteins'] - protein_reduction) / food['proteins']
                
                food['serving_size'] *= reduction_ratio
                food['carbohydrates'] *= reduction_ratio
                food['proteins'] *= reduction_ratio
                food['fats'] *= reduction_ratio

    return adjusted_meal_plan

Final Output

In [876]:
diet_type = user_input
adjusted_meal_plan_final = iterative_adjustment(meal_plan_list, allocated_macros, diet_type)

# Calculate and print the final adjusted meal plan
total_carbs, total_protein, total_fats = calculate_final_macros(adjusted_meal_plan_final)

# Calculate initial macros per meal
initial_meal_macros = calculate_macros_per_meal(adjusted_meal_plan_final)

# Calculate deltas
meal_deltas = {}
for meal, macros in initial_meal_macros.items():
    meal_deltas[meal] = {
        'Carbohydrates': allocated_macros[meal]['Carbohydrates'] - macros['Carbohydrates'],
        'Protein': allocated_macros[meal]['Protein'] - macros['Protein'],
        'Fats': allocated_macros[meal]['Fats'] - macros['Fats']
    }

adjusted_meal_plan_final = add_high_carb_food_to_meal(adjusted_meal_plan_final, high_carb_foods, meal_deltas)


# Recalculate and print final macros
total_carbs, total_protein, total_fats = calculate_final_macros(adjusted_meal_plan_final)

# Calculate excess protein
global_goal_protein = sum(allocated_macros[meal]['Protein'] for meal in allocated_macros)
excess_protein = total_protein - global_goal_protein


# Adjust serving sizes if there is excess protein
if excess_protein > 0:
    adjusted_meal_plan_final = adjust_serving_sizes_protein(adjusted_meal_plan_final, excess_protein)

# Recalculate and print final macros
total_carbs, total_protein, total_fats = calculate_final_macros(adjusted_meal_plan_final)

print("Final Suggested Meal Plan:")
for meal, foods in adjusted_meal_plan_final.items():
    print(f"\n{meal.capitalize()}:")
    for food in foods:
        print(f"  {food['product_name']} - Serving Size: {food['serving_size']:.2f}g")
        print(f"    Carbohydrates: {food['carbohydrates']:.2f}g, Proteins: {food['proteins']:.2f}g, Fats: {food['fats']:.2f}g")

print("\nFinal Total Day Macros:")
print(f"Total Day Carbohydrates: {total_carbs:.2f}g")
print(f"Total Day Protein: {total_protein:.2f}g")
print(f"Total Day Fats: {total_fats:.2f}g")


Final Suggested Meal Plan:

Breakfast:
  Preserved Duck Eggs - Serving Size: 70.58g
    Carbohydrates: 4.67g, Proteins: 16.33g, Fats: 14.00g

Lunch:
  Shrimp Wrapped In Becon - Serving Size: 123.29g
    Carbohydrates: 17.06g, Proteins: 23.89g, Fats: 15.36g

Dinner:
  Jumbo snow crab mussels peeled shrimp and fresh cut vegetables in a flavorful broth - Serving Size: 3815.51g
    Carbohydrates: 57.23g, Proteins: 57.23g, Fats: 22.00g
  Nonfat greek yogurt - Serving Size: 574.09g
    Carbohydrates: 58.16g, Proteins: 48.05g, Fats: 0.00g

Snack:
  Manzanilla Olives Stuffed With Anchovies - Serving Size: 8.96g
    Carbohydrates: 3.50g, Proteins: 10.50g, Fats: 7.00g

Final Total Day Macros:
Total Day Carbohydrates: 140.62g
Total Day Protein: 156.00g
Total Day Fats: 58.36g
